# Parameter Selection on ANFIS 
# Comparison Experiments for ANFIS and NN

In [ ]:
import torch
import numpy as np
import torch.nn as nn 
from DataSet import DataSet

## ANFIS: premise and consequent parameter selecting
from models import ANFIS_premise
from models import ANFIS_consequent


## NN + MLP
from keras.models import Sequential
from keras.layers import Dense, Activation, LSTM, CuDNNLSTM
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint

# MLR
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor


import config
import os.path as osp
import pandas as pd
import xlrd
import os
import math
import logging
import datetime



In [ ]:
class Water:
    def __init__(self, excel_files):
        self.source = DataSet(excel_files)
        
      # parameters selecting (grid search)
      
    def build_model(self, method, source):
        
        if method == 'ANFIS_PRE':
            
            pre_parameters_group, anfis_pre_rmse = self.premise_validation(source)            
            return pre_parameters_group, anfis_pre_rmse            
                
        if method == 'ANFIS_CON':
            
            anfis_final_rmse = self.consequent_validation(source, pre_parameters_group)          
            return anfis_final_rmse
        
        if method=='NN':
            
            # grid search
            nn_layer_list = [3,6,9]
            rmse_scores = []
            for k in range(len(nn_layer_list)):
                
                n, m = source.x_train.shape
                model = Sequential()
                adam = Adam(lr = 1e-3)
                
                if nn_layer_list[k]==3:
                    model.add(Dense(512, activation='relu', input_dim = m))
                    model.add(Dense(128, activation='relu'))
                    model.add(Dense(1))
                
                if nn_layer_list[k]==6
                    model.add(Dense(512, activation='relu', input_dim = m))
                    model.add(Dense(256, activation='relu'))
                    model.add(Dense(128))
                    model.add(Dense(64, activation='relu', input_dim = m))
                    model.add(Dense(32, activation='relu'))
                    model.add(Dense(1))
                    
                if nn_layer_list[k]==6
                    model.add(Dense(1024, activation='relu', input_dim = m))
                    model.add(Dense(512, activation='relu'))
                    model.add(Dense(256))
                    model.add(Dense(128, activation='relu', input_dim = m))
                    model.add(Dense(64, activation='relu'))
                    model.add(Dense(32))
                    model.add(Dense(16, activation='relu', input_dim = m))
                    model.add(Dense(8, activation='relu'))
                    model.add(Dense(4))
                    model.add(Dense(2, activation='relu', input_dim = m))
                    model.add(Dense(2, activation='relu'))
                    model.add(Dense(1))                        
                
                model.compile(loss='mean_squared_error', optimizer = adam)
                model.fit(self.source.nn_trainX, self.source.nn_trainY, 
                                                  epochs = 100,
                                                  verbose = 0
                                                 )

                pred_y = model.predict(source.nn_testX)

                nn_rmse = self.evaluate(pred_y, self.source.nn_testY, metrics)[:1]
                rmse_scores.append(nn_rmse)
                
                ## ranking
                idx = np.argsort(rmse_scores)
                nn_final_rmse = rmse_scores[idx[0]]                
            
            return nn_final_rmse
                
        if method=='MLP':
                        
            n, m = source.x_train.shape
            model = Sequential()
            model.add(Dense(512, activation='relu', input_dim = m))
            model.add(Dense(128, activation='relu'))
            model.add(Dense(1))
            adam = Adam(lr = 1e-3)
            model.compile(loss='mean_squared_error', optimizer = adam)
            model.fit(self.source.nn_trainX, self.source.nn_trainY, 
                                              epochs = 100,
                                              verbose = 0
                                             )
            
            pred_y = model.predict(source.nn_testX)
            
            mlp_rmse = self.evaluate(pred_y, self.source.nn_testY, metrics)[:1]
            
            
            return mlp_rmse  
        
        
        if method=='MLR':
            
            model = LinearRegression()  
            model = model.fit(self.source.nn_trainX, self.source.nn_trainY)
            pred_y = model.predict(source.nn_testX)
            mlr_rmse = self.evaluate(pred_y, self.source.nn_testY, metrics)[:1]
            
            return mlr_rmse
        
               
    def premise_validation(self, source):
        
        # test set
        testX = source.x_anfis_test
        testY = source.y_anfis_test
        
        # train set
        train_X = source.x_anfis_train
        train_Y = source.y_anfis_train

        inputSize = train_X.size(1)

        # init
        Anfis = ANFIS_premise(inputSize, 1).cuda().double()
        optim_anf = torch.optim.Adam(Anfis.parameters(), lr = 1e-3)
        mseLoss = nn.MSELoss()

        # other variables and hyperparameters 
        train_epoch = 100
        validation_interval == 5
        loss_sum = 0
        count = 0
        flag = 0
        best_score = 1000

        flatten_test = 20
        
        ## saving optimized premise parameters and metric
        anfis_metric = np.zeros([4])
        anfis_pre_params_group = []


        # training
        for i in range(train_epoch):                               
            optim_anf.zero_grad()
            y_hat = Anfis(train_X)                
            anfis_loss = mseLoss(y_hat, train_Y)
            anfis_loss.backward()
            optim_anf.step()


            #  validating
            if (i >= 10&& i % validation_interval==0):

                with torch.no_grad():
                    y_pred = Anfis(testX)                                                   
                    rmse_score_anf = self.evaluate(y_pred, testY, 'rmse')
                    mape_score_anf = self.evaluate(y_pred, testY, 'mape')
                    corr_score_anf = self.evaluate(y_pred, testY, 'correlation')
                    wi_score_anf = self.evaluate(y_pred, testY, 'wi')


                if rmse_score_anf < best_score:
                    # save current best
                    rmse_score_anf_cbest = rmse_score_anf
                    mape_score_anf_cbest = mape_score_anf
                    corr_score_anf_cbest = corr_score_anf
                    wi_score_anf_cbest = wi_score_anf

                    """---print and save rmse and corresponding premise parameter---"""
                    # anfis premise parameters
                    anfis_params_dic = Anfis.state_dict()
                    # all weights
                    anfis_weights_list = list(anfis_params_dic.values())
                    
                    # all premise parameters
                    anfis_pre_params = anfis_weights_list[1]
                    anfis_pre_params_group = [np.array(pre) for i, pre enumerate (anfis_pre_params) if i<10]
                    
                    
                    best_score = rmse_score_anf_cbest
                    print ("current epoch:", i)


        # average MSE/MAPE/Correlation/WI score 
        anfis_metric[0] = rmse_score_anf_cbest
        anfis_metric[1] = mape_score_anf_cbest
        anfis_metric[2] = corr_score_anf_cbest
        anfis_metric[3] = wi_score_anf_cbest

        return anfis_pre_params_group, best_score 
    
      
    def consequent_validation(self, source, pre_params_group):
        
        # test set
        testX = source.x_anfis_test
        testY = source.y_anfis_test
        
        # train set
        train_X = source.x_anfis_train
        train_Y = source.y_anfis_train

        inputSize = train_X.size(1)
        
        """hyperparameters FC layer list"""
        fc_layer_list = [1,3,6,9]
        
        
        for u in range(len(hyper_list)):
            Anfis = ANFIS_consequent(inputSize, 1, pre_params_group, fc_layer_list[u]).cuda().double()
            
            """frozen premise parameters"""
            for param in Anfis.parameters():
                param.requires_grad = True
                
            for param in model.fc1.parameters():
                param.requires_grad = False
            
            for param in model.fc2.parameters():
                param.requires_grad = False
            
            ##  only BP fc1 and fc2
            optim_anf = torch.optim.Adam(filter(lambda p: p.requires_grad, Anfis.parameters()), lr = 1e-3)
            mseLoss = nn.MSELoss()

            # other variables and hyperparameters 
            train_epoch = 100
            validation_interval == 5
            loss_sum = 0
            count = 0
            flag = 0
            best_score = 1000

            flatten_test = 20

            ## saving optimized premise parameters and metric
            anfis_metric = np.zeros([4])
            anfis_pre_params_group = []
            # all remse for all consequent parameters
            rmse_scores = [] 

            # training
            for i in range(train_epoch):                               
                optim_anf.zero_grad()
                y_hat = Anfis(train_X)                
                anfis_loss = mseLoss(y_hat, train_Y)
                anfis_loss.backward()
                optim_anf.step()


                #  validating
                if (i >= 10&& i % validation_interval==0):

                    with torch.no_grad():
                        y_pred = Anfis(testX)                                                   
                        rmse_score_anf = self.evaluate(y_pred, testY, 'rmse')
                        mape_score_anf = self.evaluate(y_pred, testY, 'mape')
                        corr_score_anf = self.evaluate(y_pred, testY, 'correlation')
                        wi_score_anf = self.evaluate(y_pred, testY, 'wi')


                    if rmse_score_anf < best_score:
                        # save current best
                        rmse_score_anf_cbest = rmse_score_anf
                        mape_score_anf_cbest = mape_score_anf
                        corr_score_anf_cbest = corr_score_anf
                        wi_score_anf_cbest = wi_score_anf

                        """---print and save rmse and corresponding premise parameter---"""
                        # anfis premise parameters
                        anfis_params_dic = Anfis.state_dict()
                        # all weights
                        anfis_weights_list = list(anfis_params_dic.values())

                        # all premise parameters
                        anfis_pre_params = anfis_weights_list[1]
                        anfis_pre_params_group = [np.array(pre) for i, pre enumerate (anfis_pre_params) if i<10]

                        best_score = rmse_score_anf_cbest
                        print ("current epoch:", i)


            # average MSE/MAPE/Correlation/WI score 
            anfis_metric[0] = rmse_score_anf_cbest
            anfis_metric[1] = mape_score_anf_cbest
            anfis_metric[2] = corr_score_anf_cbest
            anfis_metric[3] = wi_score_anf_cbest
            
            rmse_scores.append(rmse_score_anf_cbest)
            
        ## ranking
        idx = np.argsort(rmse_scores)
        best_rmse = rmse_scores[idx[0]]
                      
        return best_rmse 
    
    
        
            
    
    def evaluate(self, pred_y, true_y, metrics):
        pred_y = pred_y.reshape([-1])
        true_y = true_y.reshape([-1])
        metrics_num = len(metrics)
        results = np.zeros([metrics_num])
        for i,metric in enumerate(metrics):
            if metric == 'rmse':
                results[i] = np.sqrt(np.mean(np.power(pred_y - true_y,2)))
            elif metric == 'mape':
                results[i] = np.mean(np.abs(true_y - pred_y) / true_y)*100
            elif metric == 'correlation':
                results[i] = np.corrcoef(pred_y, true_y)[0,1]
            elif metric == 'wi':
                results[i] = np.sum(np.power(true_y - pred_y, 2)) / np.sum(np.power(np.abs(pred_y - np.mean(true_y)) + np.abs(true_y-np.mean(true_y)), 2))
        return results      
          
       
    def run_experiments(self, methods, experiments, repeat_num, metrics):
        # dic type
        results = {}
        for exp_name in experiments.keys():
            # exp_name is dict_keys type
            exp = experiments[exp_name]
            train_time, test_time = exp['times']
            evaluation = np.zeros([repeat_num, len(exp['lag_time']), len(exp['clustering']), len(exp['normalize']), len(methods), (len(metrics)-3)])
            print ('repeat_num:',repeat_num)
                        
            for r in range(repeat_num):
                for i,lag_time in enumerate(exp['lag_time']):
                    for j,clustering in enumerate(exp['clustering']):
                        for k,normalize in enumerate(exp['normalize']):
                            
                                   
                            # print and saving all ablative models
                            for t, method in enumerate(methods):
                                                                                                   
 
                                self.source.attr2source(train_time, 
                                                        normalize,
                                                        0, 
                                                        lag_time,
                                                        data_type = 'time-series')                            

                                
                                self.source.attr2source(train_time, 
                                                        normalize,
                                                        clustering, 
                                                        lag_time,
                                                        data_type = 'ts-train')
            
                                self.source.attr2source(test_time,
                                                         normalize, 
                                                         clustering, 
                                                         lag_time,
                                                         data_type = 'ts-test')
                                                
                                evaluation[r, i, j, k, t, :] = self.build_model(method, self.source) 
               
                                print('exp_name={}, lag_time={}, normalize={}, clustering={}, method={} :{}'.format(
                                    exp_name, lag_time, normalize, clustering, method, evaluation[r, i, j, k, t, :]))
                                
            # save results as a dictionary 
            np.save('result/{}.npy'.format(exp_name), evaluation)            


    def decimal2round2(self, results):
        sh = results.shape
        fraction = results.reshape(-1)
        for m in range(len(fraction)):
            fraction[m] = round(fraction[m],8)
        fractions = fraction.reshape(sh)
#         print ('fractions_shape',fractions.shape)
        return fractions 


In [ ]:
os.chdir(config_final.path)
os.environ["CUDA_VISIBLE_DEVICES"]="5"
m = Water(config_final.excel_files)
m.run_experiments(config_final.methods, 
                  config_final.experiments,
                  config_final.repeat_num,
                  config_final.metrics)